In [38]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report
from transformers import EarlyStoppingCallback
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import os

In [33]:
file_path = "/content/drive/MyDrive/PLN/EP2/ep2-train.csv"
df = pd.read_csv('./data/ep2-train.csv',
  encoding="ISO-8859-1",
  sep=";",
  decimal=",",)
df.columns = ["text", "label"]
print(df.head)

<bound method NDFrame.head of                                                     text       label
0      Sou aluna de doutorado da UFPR e pesquiso sobr...    academic
1      Gostaria de consultar a disponibilidade de cód...    academic
2      Prezados, bom dia. Procurei no site do Serpro ...  government
3      Solicito o número de matrícula do SIAP do doce...    academic
4      A Lei nº 12772/2012 regulamenta a carreira de ...    academic
...                                                  ...         ...
43673  Bom dia. gostaria de saber qual é o procedimen...     private
43674  Gostaria de saber se a Instituição possui comi...  government
43675  Gostaria de obter informação sobre o registro ...    academic
43676  Ao Ilmo. membro do Instituto Nacional do Segur...    academic
43677  Contrato CRT/DF 31.200/2012, aditivos, notas t...     private

[43678 rows x 2 columns]>


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_column = df.loc[:,'text'].str.lower()
text_column = text_column.values # ou to_numpy() – veja documentação pandas
vect = TfidfVectorizer(ngram_range=(1,1),analyzer='word')
X = vect.fit_transform(df['text']) # para uso posterior, na classificação
meuAnalyzer = vect.build_analyzer()
tokenList = []
for item in text_column:
  tokens = meuAnalyzer(item)
  tokenList.append(tokens)

In [41]:
tamanho = 300
janela = 1
minimoC = 1
caminho_modelo = "./models/word2vec/"+str(tamanho)+".kv"

# cria pasta se não existir
os.makedirs(os.path.dirname(caminho_modelo), exist_ok=True)

# verifica se o modelo existe
if os.path.exists(caminho_modelo):
    print("🔄 Carregando modelo Word2Vec existente...")
    model = Word2Vec.load(caminho_modelo.replace(".kv", ".model"))
else:
    print("⚙️  Treinando novo modelo Word2Vec...")
    model = Word2Vec(
        tokenList,
        vector_size=tamanho,
        window=janela,
        min_count=minimoC,
        sg=1,
        seed=10
    )
    # salva o modelo completo (.model) e só os vetores (.kv)
    model.save(caminho_modelo.replace(".kv", ".model"))
    model.wv.save(caminho_modelo)
    print("💾 Modelo salvo em:", caminho_modelo)

🔄 Carregando modelo Word2Vec existente...


In [42]:
def avg_document_vector(word2vec_model, doc):
  doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
  if doc:
    return np.mean(word2vec_model.wv[doc], axis=0)
  else:
    return np.zeros(word2vec_model.vector_size)

In [44]:
X_vectors = np.array([
    avg_document_vector(model, tokens)
    for tokens in tokenList
])
print(X_vectors.shape)

X_vectors_df = pd.DataFrame(X_vectors)
X_vectors_df["label"] = df["label"].values

tamanho = 300  # garante que está como int
X_vectors_df.to_csv(f'./data/X_vectors_{tamanho}.csv', index=False)

(43678, 300)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, df["label"], test_size=0.2, random_state=42, stratify=df["label"]
)

In [13]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Faz previsões
y_pred = clf.predict(X_test)

# Avalia o modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    academic       0.57      0.53      0.55      2919
  government       0.62      0.69      0.65      3756
     private       0.53      0.46      0.50      2061

    accuracy                           0.58      8736
   macro avg       0.57      0.56      0.57      8736
weighted avg       0.58      0.58      0.58      8736



In [14]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# y precisa ser numérico
le = LabelEncoder()
y_encoded = le.fit_transform(df["label"])

# separa treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# converte para tensores
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t  = torch.tensor(X_test, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.long)

# cria DataLoader
train_ds = TensorDataset(X_train_t, y_train_t)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

In [19]:
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x):
        return self.net(x)

input_dim = X_vectors.shape[1]     # 300 (tamanho do embedding)
hidden_dim = 1024                   # número de neurônios na camada oculta
output_dim = len(le.classes_)      # número de classes
model = TextClassifier(input_dim, hidden_dim, output_dim)

In [23]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Época {epoch+1}: perda média = {total_loss/len(train_dl):.4f}")

Época 1: perda média = 0.8481
Época 2: perda média = 0.8413
Época 3: perda média = 0.8374
Época 4: perda média = 0.8330
Época 5: perda média = 0.8291
Época 6: perda média = 0.8240
Época 7: perda média = 0.8192
Época 8: perda média = 0.8179
Época 9: perda média = 0.8119
Época 10: perda média = 0.8096
Época 11: perda média = 0.8048
Época 12: perda média = 0.8043
Época 13: perda média = 0.7985
Época 14: perda média = 0.7967
Época 15: perda média = 0.7952
Época 16: perda média = 0.7922
Época 17: perda média = 0.7892
Época 18: perda média = 0.7845
Época 19: perda média = 0.7823
Época 20: perda média = 0.7804
Época 21: perda média = 0.7755
Época 22: perda média = 0.7746
Época 23: perda média = 0.7697
Época 24: perda média = 0.7693
Época 25: perda média = 0.7659
Época 26: perda média = 0.7624
Época 27: perda média = 0.7621
Época 28: perda média = 0.7624
Época 29: perda média = 0.7599
Época 30: perda média = 0.7551
Época 31: perda média = 0.7536
Época 32: perda média = 0.7507
Época 33: perda m

In [26]:
model.eval()
with torch.no_grad():
    preds = model(X_test_t).argmax(dim=1)
    acc = (preds == y_test_t).float().mean().item()
    print(f"Acurácia: {acc*100:.2f}%")

Acurácia: 64.07%


In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds.numpy(), target_names=le.classes_))

              precision    recall  f1-score   support

    academic       0.65      0.63      0.64      2919
  government       0.69      0.71      0.70      3756
     private       0.54      0.54      0.54      2061

    accuracy                           0.64      8736
   macro avg       0.63      0.62      0.63      8736
weighted avg       0.64      0.64      0.64      8736

